# Advanced pattern matching and rewriting for QEC circuit rebasing

In the previous notebook, we saw how to define rewrite rules that look for
instances of a given pattern in a circuit, and replace them with a given replacement.

We will start by recapping that approach, then proceed to see how this can be used
for the use case of rebasing a logical Clifford+T circuit to make use of complex
computation primitives that arise as cheap (e.g. transversal) operations in high
distance concatenated codes.

_Note: This is currently unreleased and requires to build the tket-py crate from source in the branch `lm/pyinterface`. Activate a Python virtual environment of your choice, then `pip install maturin` and run `maturin develop` from the `./tket-py` directory._

_Note 2: I have no idea how QEC actually works, so there will be gaps I will let you
fill in for yourself._

In [1]:
from typing import List, Any

from pytket import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter

from tket._tket.ops import (
    TketOp,
)  # Note that we are importing the "wrong" TketOp, see issue https://github.com/CQCL/tket2/issues/1027
from tket.ops import TketOp as PyTketOp # This would be the "correct" TketOp
from tket.circuit import Tk2Circuit

# TODO: remove this
def matches_op(op: TketOp, op2: PyTketOp) -> bool:
    return op == op2._to_rs()

## Circuit transformation as graph rewriting

The API that we discuss in this notebook can be used to express circuit transformation
logic in terms of "rewrites", an approach that comes from graph transformation (aka rewriting) theory. You'll see -- this is cool and makes it very easy to express a wide
range of circuit transformation strategies.

There are three main concepts:

1. The `Rewriter` is an object that takes a circuit as input and returns a bunch of `CircuitRewrite` objects as outputs. This step is "read-only" in the sense that it does not modify the input circuit at all.
2. The `CircuitRewrite` objects describe potential circuit transformations that apply locally to a subcircuit of the input circuit. They can be stored and possibly combined together before being applied to the input circuit -- or they can be discarded, in which case, nothing happens to the input circuit.
3. Finally, an `Optimiser` takes the set of `CircuitRewrite` produced by one or multiple `Rewriter` objects and finds the best sequence of rewrites to apply to the input circuit. This typically happens by trial and error: rewrites are applied and their effect on the cost function is evaluated. It is then up to the optimiser to decide whether to apply further rewrites on top, or backtracking and reverting the change to apply other rewrites instead.

Let's go through a simple example to illustrate these three concepts, using the rewriters and optimisers provided out of the box by TKET.

#### 1. The ECC Rewriter

The `ECCRewriter` provides a fast way to generate rewrites that implement the typical commutation and gate cancellation rules on a given gate set. This is often useful as a basis for more complex rewrite strategies.

In [2]:
from tket.rewrite import ECCRewriter

`ECCRewriter`s rely on pre-compiled matching automata to quickly find rewrite opportunities. We can load a pre-compiled rewriter specialised for the Clifford+T gate set as follows. The `5` and `3` in the file name refer to the maximum number of gates, respectively qubit in the rewrites that will be produced.

In [3]:
rewriter = ECCRewriter.load_precompiled("../../test_files/eccs/clifford_t_5_3.rwr")

#### 2. CircuitRewrites

We can inspect the rewrites that the rewriter produces on a simple circuit:

In [4]:
circ = Circuit(2).CX(0, 1).S(0).CX(0, 1)

# Print the right-hand side of the rewrites, i.e. the new
# subcircuit fragments that the rewrite introduces.
[
    rw.replacement().to_tket1()
    for rw in rewriter.get_rewrites(Tk2Circuit(circ))
]

[[S q[0]; CX q[0], q[1]; ], [CX q[0], q[1]; S q[0]; ]]

We see that one rewrite moves the `S` gate to the left of the `CX` gate, while the other one moves it to the right.

#### 3. Optimiser

In the example above, we see that it would be beneficial to move the `S` gate out of the `CX` sandwich, so that we could then in a second step cancel out the two `CX` gates.

This is what optimisers are for! You provide them with a cost function, and they will try to find the best sequence of rewrites to apply to the input circuit to minimise the cost.

Currrently, the only optimiser provided by TKET is `badger`. Its older sibling `seadog` will be available soon!

In [5]:
from tket.optimiser import BadgerOptimiser

# we use the default cost function, which is the number of CX gates
opt = BadgerOptimiser(rewriter, cost_fn=None)
opt_circ = opt.optimise(Tk2Circuit(circ))

render_circuit_jupyter(opt_circ.to_tket1())


Excellent! We have successfully moved the `S` gate out of the `CX` sandwich, and cancelled out the two `CX` gates.

We are now ready to see how to implement custom rewriters.

## Matchers and replacers

Rewriters can be built by combining two components:

- A `CircuitMatcher`: an object that defines what patterns to match. It takes operations matched so far and a new operation and must decide if the new operation should be added to the match.
- A `CircuitReplacer`: an object that returns all possible replacements for a matched circuit. Given a matched pattern, it should return a list of circuits that the optimiser should consider as a right hand side for the rewrite.

To create such matcher and replacer objects in Python, we must implement the API given by two Python protocols,
that can be imported from `tket.protocol`:

(A python protocol is essentially a Python base class, except that inheriting from the parent class is not mandatory.)

In [6]:
from tket.protocol import CircuitMatcher, CircuitReplacer

#### 1. Circuit matchers

Copying over the function signature from `CircuitMatcher`, the API that we must implement looks like this:

```python
class CircuitMatcher:
    def match_tket_op(
        self, op: TketOp, op_args: List[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        ...
```

To understand what these arguments mean, let's create a dummy matcher: it will match any `op`. As it does so, we can print out the `op`, `op_args` and `context` values.

In [7]:
from tket.matcher import CircuitUnit, MatchContext, MatchOutcome

class AnyOpMatcher(CircuitMatcher):
    def match_tket_op(
        self, op: TketOp, op_args: List[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        print(f"op = {op}, op_args = {op_args}, context = {context}")
        return {"complete": True}

Here we already see what the return value `MatchOutcome` that we must supply should look like: it's a dictionary, in this case with a single key `complete` set to `True`.

The full type of `MatchOutcome` is defined in `tket/matcher.py`:

```python
class MatchOutcome(TypedDict, total=False):
    """
    Outcome of a pattern match.
    """

    complete: Any
    proceed: Union[Any, bool]
    skip: Union[Any, bool]
```

which is fancy Python to say that `MatchOutcome` is a dictionary that can contain any subset of the keys `complete`, `proceed` and `skip`.

To actually see the matcher in action, we must
1. combine it with a replacer
2. create a rewriter from the combination of `AnyOpMatcher` and the replacer
3. pass the rewriter to the optimiser and let it run!

A good replacer that we can use here is `ReplaceWithIdentity`: whatever circuit is matched, this replacer will result in rewrites that replace the match with the identity circuit.

In [8]:
from tket.matcher import ReplaceWithIdentity, MatchReplaceRewriter

rewriter = MatchReplaceRewriter(AnyOpMatcher(), ReplaceWithIdentity())

circ = Circuit(2).CX(0, 1).S(0).T(1).CX(0, 1)

opt = BadgerOptimiser(rewriter)
opt_circ = opt.optimise(circ)

print("number of gates:", opt_circ.n_gates)

op = CX, op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)], context = {'match_info': None, 'subcircuit': <builtins.Tk2Circuit object at 0x10770c5b0>, 'op_node': 'Node(4)'}
op = T, op_args = [CircuitUnit(linear_index=1)], context = {'match_info': None, 'subcircuit': <builtins.Tk2Circuit object at 0x103c8ff10>, 'op_node': 'Node(6)'}
op = S, op_args = [CircuitUnit(linear_index=0)], context = {'match_info': None, 'subcircuit': <builtins.Tk2Circuit object at 0x103c8ff10>, 'op_node': 'Node(5)'}
op = CX, op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)], context = {'match_info': None, 'subcircuit': <builtins.Tk2Circuit object at 0x103c8ff10>, 'op_node': 'Node(7)'}
op = CX, op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)], context = {'match_info': None, 'subcircuit': <builtins.Tk2Circuit object at 0x11f9114b0>, 'op_node': 'Node(4)'}
op = CX, op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)], context = {'match_info': N

Yeah, so it turns out that if we let any op be replaced with the identity, then the optimiser will just remove all the gates! Probably not what we want in practice 🙃

However, it's achieved our objective: we can examine the contents of `op`, `op_args` and `context` that are passed to our matcher by the optimiser.

- `op` is the least surprising: it's the operation that we are matching.
- `op_args` are the arguments passed to the operation: because our gates do not take any parameters (say a rotation angle for instance), this list contains exactly one element for each qubit of the `op`. Currently, these op_args only contain one field, `op_args[i].linear_index`, which gives us an integer that uniquely identifies the qubit on which the operation acts. We will see later that in some situation more information is available.
- `context` is a dictionary that contains a bunch more information about what we are matching. Among others, we have `subcircuit`, which contains the circuit that has been matched so far.

---

Have you noticed what's missing in our example? The matches our matcher produces are always composed of a single operation! We can see that clearly if we modify `AnyOpMatcher` to print the number of operations in `context["subcircuit"]`:

In [9]:
class AnyOpMatcher(CircuitMatcher):
    def match_tket_op(
        self, op: TketOp, op_args: List[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        print(f"num operations matched so far: {context['subcircuit'].to_tket1().n_gates}")
        return {"complete": True}

circ = Circuit(2).CX(0, 1).S(0).T(1).CX(0, 1)

rewriter = MatchReplaceRewriter(AnyOpMatcher(), ReplaceWithIdentity())
opt = BadgerOptimiser(rewriter)
opt_circ = opt.optimise(circ)

num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0
num operations matched so far: 0


Not great... What's the issue? Instead of returning `{"complete": True}`, we should return a `MatchOutcome` that indicates to the optimiser that matching should proceed further and is NOT complete yet.

We can for instance match subcircuits composed of two CX gates as follows:

In [10]:
class AnyTwoCXMatcher(CircuitMatcher):
    def match_tket_op(
        self, op: TketOp, op_args: List[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        if not matches_op(op, PyTketOp.CX):
            # We are not interested in non-CX gates
            return

        n_gates_matched = context["subcircuit"].to_tket1().n_gates
        if n_gates_matched == 0:
            # This is the first CX we matched, so we proceed to match the second.
            return {"proceed": True}
        else:
            assert n_gates_matched == 1
            # This is the second CX we matched, so we are done!
            return {"complete": True}

circ = Circuit(2).CX(0, 1).S(0).T(1).CX(0, 1).CX(0, 1)

rewriter = MatchReplaceRewriter(AnyTwoCXMatcher(), ReplaceWithIdentity())
opt = BadgerOptimiser(rewriter)
opt_circ = opt.optimise(circ)

render_circuit_jupyter(opt_circ)

Hey, this is actually a semantically correct optimisation! We are removing any pair of CX gates.

However, this has issues: it will for instance remove two CX even if the second is upside down, or if it does not act on the same qubits:

In [11]:
circ = Circuit(2).CX(0, 1).CX(1, 0)
assert opt.optimise(circ).n_gates == 0 # not great...

circ = Circuit(3).CX(0, 1).CX(1, 2)
assert opt.optimise(circ).n_gates == 0 # arguably worse!


To resolve this, we should track state throughout our matching process:
- when matching the first CX, we should remember the qubits it acts on
- when matching the second CX, we should only match successfully if it acts on the same qubits as the first CX

We can do this by returning a (non-boolean!) value in our MatchOutcome dict:
```python
return {"proceed": [ctrl_qubit, tgt_qubit]}
```

This value is then passed back to us when we match the second CX, within `context["match_info"]`. This gives the following:

In [12]:
class TwoCXMatcher(CircuitMatcher):
    def match_tket_op(
        self, op: TketOp, op_args: list[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        if not matches_op(op, PyTketOp.CX):
            return

        # use the `match_info` dict key to track the qubit IDs matched previously
        prev_matched_qubits = context["match_info"]

        match prev_matched_qubits:
            case None:
                # This is the first CX we matched, so we proceed to match the second.
                qubits = [arg.linear_index for arg in op_args]
                return {"proceed": qubits}
            case [ctrl_qubit, tgt_qubit]:
                curr_qubits = [arg.linear_index for arg in op_args]
                if curr_qubits == [ctrl_qubit, tgt_qubit]:
                    # We have successfully matched two CXs, so we are done!
                    return {"complete": True}
                else:
                    # The second CX is upside down, so we are not interested.
                    return


circ = Circuit(3).CX(0, 1).CX(1, 0).CX(1, 0).CX(0, 2)
rewriter = MatchReplaceRewriter(TwoCXMatcher(), ReplaceWithIdentity())
opt = BadgerOptimiser(rewriter)
opt_circ = opt.optimise(circ)

render_circuit_jupyter(opt_circ)

This has indeed left the CX that were not on the same qubits unchanged!

Passing around data as `match_info` is super powerful: you are free to use it for whatever you want. Instead of printing out from the matcher code directly (which interleaves all the various matching operations that the optimiser might perform in parallel), we could for instance keep a trace of the matching process within `match_info`. This would make debugging a lot easier!

In [13]:
class TwoCXMatcher(CircuitMatcher):
    def match_tket_op(
        self, op: TketOp, op_args: list[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        if not matches_op(op, PyTketOp.CX):
            return

        # use the `match_info` dict key to track the qubit IDs matched previously
        prev_matched_qubits = context["match_info"]

        match prev_matched_qubits:
            case None:
                # This is the first CX we matched, so we proceed to match the second.
                qubits = [arg.linear_index for arg in op_args]
                debug_info = f"matched first CX: op_args = {op_args}\n"
                return {"proceed": (qubits, debug_info)}
            case ([ctrl_qubit, tgt_qubit], debug_info):
                curr_qubits = [arg.linear_index for arg in op_args]
                if curr_qubits == [ctrl_qubit, tgt_qubit]:
                    # We have successfully matched two CXs, so we are done!
                    debug_info += f"matched second CX: op_args = {op_args}\n"
                    return {"complete": debug_info}
                else:
                    # The second CX is upside down, so we are not interested.
                    return

Now, the issue is that if we were to run this we would not see any of the debug information as it never gets printed out. However, the code above contains a subtle change: we are now returning a string from the `complete` key, instead of the previous boolean.

In complete analogy to passing a value to the `proceed` key, we can also pass a value to the `complete` key. This value won't appear in future calls to `match_tket_op`, for the simple reason that there will not be any such future calls (the matching process has completed!). It is instead passed to the replacer.

It is thus time to implement our own replacer.

#### 2. Circuit replacer

The API, copied over from `tket/protocol.py`, that we need to implement looks as follows:

```python
class CircuitReplacer(Protocol):
    def replace_match(self, circuit: Tk2Circuit, match_info: Any) -> List[Tk2Circuit]:
        ...
```

As you can see, there is a `match_info` argument! That is precisely where we will find the value that we have passed to the `complete` key in the `MatchOutcome` dictionary. Here's a simple replacer that returns an empty two-qubit circuit. We then run the matcher-replacer pair using Badger, which allows us to see the debug information for each successful match of the matcher:

In [14]:
class EmptyTwoQubitReplacer(CircuitReplacer):
    def replace_match(self, circuit: Tk2Circuit, match_info: Any) -> list[Tk2Circuit]:
        print(f"match_info = {match_info}")
        return [Tk2Circuit(Circuit(2))]

circ = Circuit(2).CX(0, 1).CX(0, 1).CX(1, 0).CX(1, 0)
rewriter = MatchReplaceRewriter(TwoCXMatcher(), EmptyTwoQubitReplacer())
opt = BadgerOptimiser(rewriter)
opt_circ = opt.optimise(circ)

match_info = matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]
matched second CX: op_args = [CircuitUnit(linear_index=0, linear_pos=before), CircuitUnit(linear_index=1, linear_pos=before)]

match_info = matched first CX: op_args = [CircuitUnit(linear_index=1), CircuitUnit(linear_index=0)]
matched second CX: op_args = [CircuitUnit(linear_index=1, linear_pos=before), CircuitUnit(linear_index=0, linear_pos=before)]

match_info = matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]
matched second CX: op_args = [CircuitUnit(linear_index=0, linear_pos=after), CircuitUnit(linear_index=1, linear_pos=after)]

match_info = matched first CX: op_args = [CircuitUnit(linear_index=1), CircuitUnit(linear_index=0)]
matched second CX: op_args = [CircuitUnit(linear_index=1, linear_pos=before), CircuitUnit(linear_index=0, linear_pos=before)]

match_info = matched first CX: op_args = [CircuitUnit(linear_index=1), CircuitUnit(linear_inde

Perfect! This will be great for debugging.

Speaking of which: we can see from the matches we get that there seem to be duplicate matches: we see six matches when there really should be only two. This is not an issue for correctness, the optimiser will always remove the duplicates, but it's not great for performance.

Upon closer inspection, we see there is a new piece of data showing up in the `op_args` arguments when matching the second CX: the `op_args[i].linear_pos` field. This field contains information about the relative position of the operation with respect to previously matched operations. This only exists of course when operations have matched on the same qubit, which is why we didn't see it before. Using this data, we can fix our matcher to no longer produce duplicate matches:

In [ ]:
class TwoCXMatcher(CircuitMatcher):
    def match_tket_op(
        self, op: TketOp, op_args: list[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        if not matches_op(op, PyTketOp.CX):
            return

        # use the `match_info` dict key to track the qubit IDs matched previously
        prev_matched_qubits = context["match_info"]

        match prev_matched_qubits:
            case None:
                # This is the first CX we matched, so we proceed to match the second.
                qubits = [arg.linear_index for arg in op_args]
                debug_info = f"matched first CX: op_args = {op_args}\n"
                return {"proceed": (qubits, debug_info)}
            case ([ctrl_qubit, tgt_qubit], debug_info):
                # Only match the second CX if it comes after the first CX on all qubits
                if not all(arg.linear_pos == "after" for arg in op_args):
                    return

                curr_qubits = [arg.linear_index for arg in op_args]
                if curr_qubits == [ctrl_qubit, tgt_qubit]:
                    # We have successfully matched two CXs, so we are done!
                    debug_info += f"matched second CX: op_args = {op_args}\n"
                    return {"complete": debug_info}
                else:
                    # The second CX is upside down, so we are not interested.
                    return

circ = Circuit(2).CX(0, 1).CX(0, 1).CX(1, 0).CX(1, 0)
rewriter = MatchReplaceRewriter(TwoCXMatcher(), EmptyTwoQubitReplacer())
opt = BadgerOptimiser(rewriter)
opt_circ = opt.optimise(circ)

matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]

matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]

matched first CX: op_args = [CircuitUnit(linear_index=1), CircuitUnit(linear_index=0)]

matched first CX: op_args = [CircuitUnit(linear_index=1), CircuitUnit(linear_index=0)]

[CircuitUnit(linear_index=0, linear_pos=before), CircuitUnit(linear_index=1, linear_pos=before)]
not all after
[CircuitUnit(linear_index=0, linear_pos=before), CircuitUnit(linear_index=1, linear_pos=before)]
not all after
[CircuitUnit(linear_index=1, linear_pos=before), CircuitUnit(linear_index=0, linear_pos=before)]
not all after
matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]
matched second CX: op_args = [CircuitUnit(linear_index=0, linear_pos=after), CircuitUnit(linear_index=1, linear_pos=after)]

match_info = matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]
mat

Bingo!

### Advanced features of matchers and replacers

#### 1. Matching extension ops

The `match_tket_op` method will not work on ops that TKET does not recognise. By default, if an op that is not a typical TKET gate is encountered, it will be silently ignored.

To handle this case, the Matcher API provides the `match_extension_op` method:

```python
class CircuitMatcher:
    ... 

    def match_extension_op(
        self,
        op: CustomOp,
        inputs: List[CircuitUnit],
        outputs: List[CircuitUnit],
        context: MatchContext,
    ) -> MatchOutcome:
```

This method is called instead of `match_tket_op` if the op is not a typical TKET gate and the optype is passed as a `tket.ops.CustomOp`. Matching of such custom ops can be done using the name property of `CustomOp`. Note that unlike `TketOp`s, for which a single list of arguments is passed, when matching `CustomOp`s, the matcher API distinguishes between `inputs` and `outputs` arguments (as unlike typical quantum gates, there is no guarantee that they are identical).

I highly recommend checking out the docs of `tket.protocol.CircuitMatcher` for more details on this API.

#### 2. Combining multiple matches to create a single rewrite

It's important to keep in mind some of the limitations/design choices of the Matcher API. The Matcher will start by matching a single operation, and then attempt to expand a match to form a larger subcircuit, appending new operations to the beginning or the end of the previously matched subcircuit. This means in particular:

1. The Matcher API only supports matching _connected_ subcircuits: from a single matching operation, an entire matched subcircuit is obtained by gradually expanding the boundary of the matched region.
2. The matcher API will only let you match convex subcircuits: the optimiser will never extend a matched subcircuit to an additional operation if this would violate convexity. This gives you a guarantee that operations will be matched in a certain order (there won't be "gaps" during the matching process), but it is of course limiting in the expressivity of the matcher.

There is a specific rewriter designed to alleviate the first limitation above: `CombineMatchReplaceRewriter`:

In [60]:
from tket.matcher import CombineMatchReplaceRewriter

To recap, this brings the list of rewriters that we have seen to three:

- `ECCRewriter`: generates rewrites that apply the typical commutation and gate cancellation rules for a given gate set.
- `MatchReplaceRewriter`: combines a matcher and a replacer to create a new rewriter. The matcher and replacer can be chosen from the TKET-provided set of matchers and replacers, or custom ones can be implemented using the Matcher and Replacer APIs.
- `CombineMatchReplaceRewriter`: similar to `MatchReplaceRewriter`, but allows you to combine multiple matchers to form subcircuit matches that are disconnected. The union of the matched subcircuits of the individual matchers will form the overall matched subcircuit that is passed to the replacer and used to construct the rewrite.

#### 3. Using a custom cost function in `BadgerOptimiser`

The optimiser used the default cost function so far, which is the number of CX gates. We can pass a custom cost function to be minimised. The cost function should be integer-valued, so a natural solution is to define a dict mapping op types to their cost.

```python
op_costs = { PyTketOp.CX: 1, PyTketOp.H: 2 }
opt = BadgerOptimiser(transversal_hadamard, cost_fn=lambda op: op_costs[op.type])
```

#### Example: Matching transversal Hadamards

Let's say we are interested in pairing up Hadamard gates if they appear on qubits 2k and 2k+1 for some k. We can implement this by combining a matcher matching Hadamard on even qubits with a matcher matching Hadamard on odd qubits:

In [61]:
class HadamardMatcher(CircuitMatcher):
    match_even: bool

    def __init__(self, match_even: bool):
        self.match_even = match_even

    def match_tket_op(self, op: TketOp, op_args: list[CircuitUnit], context: MatchContext) -> MatchOutcome:
        if not matches_op(op, PyTketOp.H):
            return

        [qubit] = [arg.linear_index for arg in op_args]
        if qubit % 2 == 0 and self.match_even:
            return {"complete": qubit}
        elif qubit % 2 == 1 and not self.match_even:
            return {"complete": qubit}

matcher_pair = [HadamardMatcher(True), HadamardMatcher(False)]

Now, in the replacer, we will get all pairs of Hadamards such that one was applied on an even qubit and the other on an odd qubit. We need to throw away the pairs that are not on adjacent qubits. We can do this by returning an empty list of rewrites in the non-adjacent case.

This is as good a moment as any to observe that you can return as many (or as few) replacement circuits in the CircuitReplacer's `get_rewrites` method as you want: returning an empty list means that there is no valid substitution for the given match, returning multiple replacements means the optimiser must choose which one (if any) to apply.

I don't know how to create custom op types at the moment, so I will just use a `CZ` gate as a placeholder for a two-qubit transversal Hadamard 😅

In [62]:
class TransversalHadamardReplacer(CircuitReplacer):
    def replace_match(self, circuit: Tk2Circuit, match_info: Any) -> list[Tk2Circuit]:
        # the two values returned by the matchers are passed in match_info
        [qubit1, qubit2] = match_info

        if qubit1 + 1 == qubit2:
            return [Tk2Circuit(Circuit(2).CZ(0, 1))]
        else:
            return []

transversal_hadamard = CombineMatchReplaceRewriter(matcher_pair, TransversalHadamardReplacer())

circ = Circuit(4).H(0).H(1).H(2).H(3).H(0).CX(0, 1).H(1)

opt = BadgerOptimiser(transversal_hadamard, cost_fn=lambda op: 0 if matches_op(op, PyTketOp.CZ) else 1 )
opt_circ = opt.optimise(circ)

render_circuit_jupyter(opt_circ)




To make the optimiser perform the rewrites (and introduce the new CZ gates), we had to change the cost function: we made CZ gates free, and every other gate cost 1 (unfortunately negative costs are not supported at the moment).

Finally, note that the remaining two Hadamards were not matched up: that is because they are split by a CX gate, and thus the matched subcircuit would not be convex (it would also lead to an invalid circuit if the two Hadamards were replaced by a CZ gate).

### A fully fledged example

I am copying over the example from the previous notebook here, in case having a fully working example is useful for getting started.
The only "interesting" thing about this example is that mulitple rewriters are used in parallel, by defining the optimiser as
```python
opt = BadgerOptimiser([cancel_cx, flip_zzphase])
```

---

To complete this notebook, we will implement a slightly more interesting optimisation, which combines CX cancellation as above with ZZPhase flipping.

The `flip_zzphase` rewriter is composed of the `ZZPhaseMatcher` and `FlippedZZPhase` replacer:

In [63]:
from typing import Literal

def succeeds_previous_op(op_args: list[CircuitUnit]) -> bool:
    """Whether this current op is in the future of previously matched ops."""
    return all(arg.linear_pos in ["after", None] for arg in op_args)

# This reuses the TwoCXMatcher from above
cancel_cx = MatchReplaceRewriter(TwoCXMatcher(), ReplaceWithIdentity())

MatchState = None \
    | tuple[Literal["matched_first_cx"], int, int]\
    | tuple[Literal["matched_rotation"], int, int, float]

class ZZPhaseMatcher(CircuitMatcher):
    def match_tket_op(
        self, op: TketOp, op_args: list[CircuitUnit], context: MatchContext
    ) -> MatchOutcome:
        state: MatchState = context["match_info"]

        match state:
            case None:
                # We are looking for a CX
                if matches_op(op, PyTketOp.CX):
                    # This is the first matched op, so the relative position of
                    # the qubits with respect to the already matched subcircuit
                    # is not yet known.
                    [ctrl_qubit, tgt_qubit] = [arg.linear_index for arg in op_args]
                    assert all(arg.linear_pos is None for arg in op_args)

                    return {
                        "proceed": ("matched_first_cx", ctrl_qubit, tgt_qubit)
                    }
                else:
                    return { "stop": True }
            case ("matched_first_cx", ctrl_qubit, tgt_qubit):
                # must come after the first CX
                if not succeeds_previous_op(op_args):
                    return { "skip": True }

                # We are looking for a rotation
                if matches_op(op, PyTketOp.Rz) and op_args[0].linear_index == tgt_qubit \
                                               and op_args[1].constant_float is not None:
                    rot_angle = op_args[1].constant_float
                    return {
                        "proceed": ("matched_rotation", ctrl_qubit, tgt_qubit, rot_angle)
                    }
                else:
                    return { "skip": True }
            case ("matched_rotation", ctrl_qubit, tgt_qubit, rot_angle):
                # must come after the first CX and the rotation
                if not succeeds_previous_op(op_args):
                    return { "skip": True }

                # We are looking for a second CX
                if matches_op(op, PyTketOp.CX) and [arg.linear_index for arg in op_args] == [ctrl_qubit, tgt_qubit]:
                    return { "complete": rot_angle }
                else:
                    return { "skip": True }

class FlippedZZPhase(CircuitReplacer):
    def replace_match(self, circuit: Tk2Circuit, match_info: float) -> list[Tk2Circuit]:
        assert circuit.to_tket1().n_qubits == 2

        flipped_circ = Circuit(2).CX(1, 0).Rz(match_info, 0).CX(1, 0)
        # zzphase_circ = Circuit(2).CX(0, 1).Rz(match_info, 1).CX(0, 1)
        return [Tk2Circuit(flipped_circ)]

assert isinstance(ZZPhaseMatcher(), CircuitMatcher)
assert isinstance(FlippedZZPhase(), CircuitReplacer)

flip_zzphase = MatchReplaceRewriter(ZZPhaseMatcher(), FlippedZZPhase())

We would like to combine `flip_zzphase` with `cancel_cx` to optimise the following circuit:

In [64]:
circ = Circuit(3).CX(0, 1).CX(0, 1).Rz(0.111, 1).CX(0, 1).CX(2, 0).CX(0, 2).Rz(0.5, 2).CX(0, 2)

assert len(flip_zzphase.get_rewrites(Tk2Circuit(circ))) == 2

render_circuit_jupyter(circ)

Note that the order in which the rewrites should be performed is important in this case: if we apply all zzphase flips first, then the first two CX gates will no longer cancel out.

The badger optimiser is smart: it will always find the best sequence of rewrites to minimise CX count. In this case, it will thus cancel the first two CX gates, then flip the ZZPhase at the end of the circuit before cancelling the resulting two CXs in the middle of the circuit:

In [65]:
opt = BadgerOptimiser([cancel_cx, flip_zzphase])
opt_circ = opt.optimise(circ)

# assert opt_circ.n_gates_of_type(OpType.CX) == 2

render_circuit_jupyter(opt_circ)

matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]

matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]

matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]

matched first CX: op_args = [CircuitUnit(linear_index=2), CircuitUnit(linear_index=0)]

matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=2)]

matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=2)]

[CircuitUnit(linear_index=0, linear_pos=before), CircuitUnit(linear_index=1, linear_pos=before)]
not all after
[CircuitUnit(linear_index=2, linear_pos=before), CircuitUnit(linear_index=0, linear_pos=before)]
not all after
[CircuitUnit(linear_index=0, linear_pos=before), CircuitUnit(linear_index=1)]
not all after
matched first CX: op_args = [CircuitUnit(linear_index=0), CircuitUnit(linear_index=1)]
matched second CX: op_args = [CircuitUnit(linear_index=0, linear_pos=af